In [1]:
import numpy as np
import os, re, csv, math, codecs
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer as t
from tqdm import tqdm
from keras.layers import merge, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import *
# to visualize, and to make zero shape matrix
from attention_utils import get_activations, get_data_recurrent
# from Attention import Attention
import pandas as pd
from konlpy.tag import Okt as Twitter
# from selfword2vec import tokenization
# from Anomaly import checkAnomaly_x_y
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from numpy import argmax


# 1. 모델 저장시키기 
# 2. tokenizer와 konlpy morphs 호환 여부 (완성 jupyter note)
# 2'. word2vec 2가지 경우 더 추가 to embedding(final_total word2vec, twitter_translated.vec)
# 3. 변수들 설정하기


np.random.seed(3)

# NUM_WORDS, train으로 input받은 단어의 수
MAX_NB_WORDS = 20000
vocab_size = 0
EMB_DIM = 300
embeddings_index = dict()

# columns = ["ID", "Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism","sadness","surprise","trust"]
columns = ["ID","Tweet","분노","기대","혐오스러운","두려움","기쁨","사랑","낙관론","비관론","슬픔","놀라움","믿음"]




##########################################1. 데이터셋 생성하기

train_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_train.txt",sep="\t", header=None,names=columns).values
val_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_dev.txt",sep="\t", header=None,names=columns).values
test_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_test_gold.txt",sep="\t", header=None,names=columns).values

# 판다 shape
print("train_array"+ str(train_array.shape))
print("val_array"+str(val_array.shape))
print("test_array"+str(test_array.shape))

print("Reading data!")
#  x, y 분할하기
x_train = train_array[1:,1]
y_train = train_array[1:,2:]
x_val = val_array[1:,1]
y_val = val_array[1:,2:]
x_test = test_array[1:,1]
y_test = test_array[1:,2:]

print("ANomaly result!")
print("x_train.shape"+ str(x_train.shape))
print("y_train.shape"+ str(y_train.shape))
print("x_val.shape"+ str(x_val.shape))
print("y_val.shape"+ str(y_val.shape))
print("x_test.shape"+ str(x_test.shape))
print("y_test.shape"+ str(y_test.shape))
# print(type(x_train)) np.array로 변형이 필요한가
print("Finished!")

# x_train[1]
type(y_train[0][0])

Using TensorFlow backend.


train_array(6651, 13)
val_array(887, 13)
test_array(3239, 13)
Reading data!
ANomaly result!
x_train.shape(6650,)
y_train.shape(6650, 11)
x_val.shape(886,)
y_val.shape(886, 11)
x_test.shape(3238,)
y_test.shape(3238, 11)
Finished!


str

In [2]:
trimed_x = list()
trimed_y = list()
# trimed_y = np.ndarray()
for i,y in enumerate(y_train):
    if len(y) == 11:
        sent = list()
        for j,f in enumerate(y):
            try:
                float(f)
                sent.append(f)
            except ValueError:
                #value가 float화 되지 못한다면 다음 문장으로 넘어간다.
                sent = list()
                break
        if sent:
            trimed_x.append(x_train[i])
            trimed_y.append(y_train[i])

print(np.array(trimed_x).shape)
print(np.array(trimed_y).shape)


# return np.array(trimed_x), np.array(trimed_y)
        
        
        
        

(6619,)
(6619, 11)


In [7]:
y_train = np.array(trimed_y).astype('float32')

In [10]:
y_train[0]

array([ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.], dtype=float32)

In [69]:
j = np.array([1,2,3])
k = list()

In [70]:
k.extend(a)

In [62]:
k
type(k[0])

numpy.int64

In [72]:
if np.array(k).shape == j.shape:
    print("fu")

fu


In [41]:
from sklearn.metrics import jaccard_similarity_score
threshold =0.5
y_pred = np.array([0.08,0.03,0.5,1]).astype('float32')
y_pred = y_pred.round()
y_true = np.array([1.0,1.0,0.0,0.0])
jaccard_similarity_score(y_true,y_pred)


0.25

In [38]:
y_pred

array([ 1.,  0.,  0.,  1.], dtype=float32)

In [149]:
from keras import backend as K
threshold = 0.6
y_pred = np.array([0.59,1,0.59,0.4])
y_pred = (y_pred>threshold).astype('float64')
y_true = np.array([1.0,1.0,0.0,0.0])

intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
jac = (intersection) / (sum_ - intersection)

In [127]:
print(intersection)

Tensor("Sum_29:0", shape=(), dtype=float64)


In [128]:
print(sum_)

Tensor("Sum_30:0", shape=(), dtype=float64)


In [129]:
print(jac)

Tensor("truediv_14:0", shape=(), dtype=float64)


In [151]:
K.eval(intersection)

1.0

In [152]:
K.eval(jac)

0.5

In [153]:
jaccard_similarity_score(y_true,y_pred)

0.75

In [154]:
import sys
def tanimoto_coefficient(p_vec, q_vec):
    e = sys.float_info.epsilon
    pq = p_vec * q_vec
    p_square = np.sqrt(p_vec)
    q_square = np.sqrt(q_vec)
    return pq / (p_square + q_square - pq+e)

In [155]:
tanimoto_coefficient(y_true,y_pred)

array([ 0.,  1.,  0.,  0.])